In [1]:
from qdrant_client import QdrantClient
import os

qclient = QdrantClient(
    url=os.getenv('QDRANT_DB_URL'),
    api_key=os.getenv('QDRANT_API_KEY')
)

qclient

In [2]:
from qdrant_client.http.models import Filter, FieldCondition, MatchValue
import os
from tqdm import tqdm 

In [3]:
collection_name = "images_and_clothes_swin"

In [4]:
scroll_result = qclient.scroll(
    collection_name=collection_name,
    with_payload=False,  # we just need the IDs
    limit=10000,         # change if you have more points
)

all_point_ids = [point.id for point in scroll_result[0]]

In [10]:
for point_id in tqdm(all_point_ids, desc="Updating payloads"):
    qclient.set_payload(
        collection_name=collection_name,
        payload={
            "bought": False,
            "not_interested": False
        },
        points=[point_id]
    )

Updating payloads:  70%|██████▉   | 3018/4313 [31:09<13:22,  1.61it/s]  


ResponseHandlingException: The read operation timed out

In [9]:
limit = 1000  # scroll batch size
offset = None
all_to_fix = []

while True:
    points, next_page = qclient.scroll(
        collection_name=collection_name,
        with_payload=True,
        limit=limit,
        offset=offset
    )

    for pt in points:
        if "not interested" in pt.payload:
            all_to_fix.append(pt.id)

    if next_page is None:
        break
    offset = next_page

# Now delete the bad payload key from all affected points
if all_to_fix:
    print(f"Deleting 'not interested' from {len(all_to_fix)} points...")

    # Qdrant accepts max 65536 IDs in one call, so batch it
    for i in range(0, len(all_to_fix), 5000):
        qclient.delete_payload(
            collection_name=collection_name,
            points=all_to_fix[i:i + 5000],
            keys=["not interested"]
        )
    print("✅ Done.")
else:
    print("No points had the incorrect key.")

Deleting 'not interested' from 358 points...


UnexpectedResponse: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Format error in JSON body: Invalid json path: \'not interested\' at line 1 column 26"},"time":0.0}'

In [11]:
update_filter = Filter(
    should=[
        FieldCondition(key="bought", match=MatchValue(value=True)),
        FieldCondition(key="not_interested", match=MatchValue(value=True)),
    ]
    # 'min_should=1' is the default, so it's an OR-clause.
)

# ✅ Use set_payload to reset both flags on _every_ point matching the filter:
result = qclient.set_payload(
    collection_name=collection_name,
    payload={"bought": False, "not_interested": False},
    points=update_filter,
    wait=True,  # Optional: block until operation is committed
)

print("Updated points:", result)  # prints acknowledgment & operation info

Updated points: operation_id=3703 status=<UpdateStatus.COMPLETED: 'completed'>
